# Level 2: Hierarchical RAG

This notebook will show you how to build a RAG application with Llama Stack using multi-field document indexing. You will learn how the API's provided by Llama Stack can be used to directly control and invoke all common RAG stages, including indexing, retrieval and inference. 

In this example, we'll work with IT call center tickets and combine multiple fields (`short_description`, `content`, and `close_notes`) to create richer document representations that improve retrieval quality and context understanding.


## Overview

This tutorial covers the following steps:
1. Loading and preparing the dataset, combining multiple fields to create comprehensive document content.
2. Indexing the enriched documents into a vector database (ChromaDB) for later retrieval.
3. Executing the built-in RAG tool to retrieve the document chunks relevant to a given query.
4. Using the retrieved context to answer user queries during the inference step.
5. Understanding why multi-field RAG outperforms single-field RAG through practical query examples.

## 1. Setting Up this Notebook

First, we will start with a few imports.

In [ ]:
import uuid

from llama_stack_client import RAGDocument
from llama_stack_client.types import Document
from llama_stack_client.lib.agents.agent import Agent


In [ ]:
import pandas as pd
from pathlib import Path

# Load the CSV file from the data directory
data_dir = Path("../data")
file_path = data_dir / "synthetic-it-call-center-tickets.csv"

# Carrega o CSV em um DataFrame do pandas
df = pd.read_csv(file_path)

# Exibe as primeiras 5 linhas e informações do DataFrame
df.head()

In [ ]:
df.info()

Next, we will initialize our environment as described in detail in our ["Getting Started" notebook](Level0_getting_started_with_Llama_Stack.ipynb). Please refer to it for additional explanations.

In [ ]:
# Import required libraries
import os
import sys
from pathlib import Path

# Add root src directory to path to import shared config
root_dir = Path("../..").resolve()
sys.path.insert(0, str(root_dir / "src"))

# Import centralized configuration
from config import LLAMA_STACK_URL, MODEL, CONFIG

# For communication with Llama Stack
from llama_stack_client import LlamaStackClient
from termcolor import cprint

# Configuration values (automatically detected based on environment)
llamastack_url = LLAMA_STACK_URL
model = MODEL

if not llamastack_url:
    raise ValueError(
        "LLAMA_STACK_URL is not configured!\n"
        "Please run: ./scripts/setup-env.sh\n"
        "Or set LLAMA_STACK_URL environment variable:\n"
        "  export LLAMA_STACK_URL='https://llamastack-route-my-first-model.apps.ocp.example.com'"
    )

print(f"📡 LlamaStack URL: {llamastack_url}")
print(f"🤖 Model: {model}")
print(f"📍 Environment: {'Inside OpenShift cluster' if CONFIG['inside_cluster'] else 'Outside OpenShift cluster'}")
print(f"📦 Namespace: {CONFIG['namespace']}")

# Tavily search API key is optional (only needed for some demos)
tavily_search_api_key = os.getenv("TAVILY_SEARCH_API_KEY")
if tavily_search_api_key is None:
    provider_data = None
else:
    provider_data = {"tavily_search_api_key": tavily_search_api_key}

# Initialize LlamaStack client
client = LlamaStackClient(
    base_url=llamastack_url,
    provider_data=provider_data
)

# Verify connection
try:
    models = client.models.list()
    model_count = len(models.data) if hasattr(models, 'data') else len(models)
    print(f"\n✅ Connected to LlamaStack")
    print(f"   Available models: {model_count}")
except Exception as e:
    print(f"\n❌ Cannot connect to LlamaStack: {e}")
    print("\n💡 Troubleshooting:")
    print("   1. Check if route exists: oc get route llamastack-route -n my-first-model")
    print("   2. Run setup script: ./scripts/setup-env.sh")
    print("   3. Or set LLAMA_STACK_URL manually in .env file")
    raise

# model_id for the model you wish to use that is configured with the Llama Stack server
model_id = "ollama/llama3.2:3b"

temperature = float(os.getenv("TEMPERATURE", 0.0))
if temperature > 0.0:
    top_p = float(os.getenv("TOP_P", 0.95))
    strategy = {"type": "top_p", "temperature": temperature, "top_p": top_p}
else:
    strategy = {"type": "greedy"}

max_tokens = int(os.getenv("MAX_TOKENS", 4096))

# sampling_params will later be used to pass the parameters to Llama Stack Agents/Inference APIs
sampling_params = {
    "strategy": strategy,
    "max_tokens": max_tokens,
}

stream_env = os.getenv("STREAM", "True")
# the Boolean 'stream' parameter will later be passed to Llama Stack Agents/Inference APIs
# any value non equal to 'False' will be considered as 'True'
stream = (stream_env != "False")

print(f"Inference Parameters:\n\tModel: {model_id}\n\tSampling Parameters: {sampling_params}\n\tstream: {stream}")

Finally, we complete the setup by initializing the document collection we will use for RAG ingestion and retrieval.

## 2. Indexing the Documents
- Initialize a new document collection in our vector database. All parameters related to the vector database, such as the embedding model and dimension, must be specified here.
- Create RAG documents by combining multiple fields from the dataset: `short_description`, `content`, and `close_notes`. This provides richer context for better retrieval and understanding of the tickets.
- Provide the list of documents to the RAG tool. Llama Stack will handle chunking and indexing the content into the vector database.

In [ ]:
client.providers.list()

In [ ]:
client.models.list()

In [ ]:

# Explicit - specify embedding model and/or provider when you need specific ones
vs_chroma = client.vector_stores.create(
    extra_body={
        "provider_id": "chromadb",  # Optional: specify vector store provider
        "embedding_model": "sentence-transformers/nomic-ai/nomic-embed-text-v1.5",
        "embedding_dimension": 768  # Optional: will be auto-detected if not provided
    }
)

In [ ]:
df = df.fillna("")

In [ ]:
# Limita o dataset para os primeiros 1000 registros para processamento mais rápido
df_1000 = df.head(1000)

In [ ]:
# Cria documentos RAG combinando múltiplos campos para enriquecer o contexto
# Os campos 'short_description', 'content' e 'close_notes' são combinados no conteúdo principal
# Os demais campos são armazenados como metadados para filtragem e referência
documents = [
    RAGDocument(
        document_id=f"num-{i}",
        content=f"{df_1000.iloc[i]['short_description']}\n\n{df_1000.iloc[i]['content']}\n\n{df_1000.iloc[i]['close_notes']}",
        mime_type="text/plain",
        metadata=df_1000.iloc[i].drop(["short_description", "content", "close_notes"]).to_dict(),
    )
    for i in range(len(df_1000))
]

In [ ]:
documents

In [ ]:
client.tool_runtime.rag_tool.insert( 
    chunk_size_in_tokens=1024,
    documents=documents,
    vector_db_id=str(vs_chroma.id),
    extra_body={"vector_store_id": str(vs_chroma.id)},
    extra_headers=None,
    extra_query=None,
    timeout=None
)

In [ ]:
client.vector_io.query(vector_db_id=vs_chroma.id,query="ZTrend crashes")

## 3. Executing Queries via the Built-in RAG Tool
- Directly invoke the RAG tool to query the vector database we ingested into at the previous stage.
- Construct an extended prompt using the retrieved chunks.
- Query the model with the extended prompt.
- Output the reply received from the model.

In [ ]:
queries = [
    "What was the root cause and resolution for application crashes related to memory issues?",
]

for prompt in queries:
    cprint(f"\nUser> {prompt}", "blue")
    
    # RAG retrieval call
    rag_response = client.tool_runtime.rag_tool.query(
        content=prompt,
        vector_db_ids=[str(vs_chroma.id)],   # o SDK exige isso
        extra_body={"vector_store_ids": [str(vs_chroma.id)]},  # o backend exige isso
    )

    print(rag_response.content)
    # the list of messages to be sent to the model must start with the system prompt
    messages = [{"role": "system", "content": "You are a helpful assistant."}]

    # construct the actual prompt to be executed, incorporating the original query and the retrieved content
    prompt_context = rag_response.content
    extended_prompt = f"Please answer the given query using the context below.\n\nCONTEXT:\n{prompt_context}\n\nQUERY:\n{prompt}"
    messages.append({"role": "user", "content": extended_prompt})

    # use Llama Stack inference API to directly communicate with the desired model
    response = client.chat.completions.create(
        messages=messages,
        model=model,
        stream=stream,
        max_tokens=max_tokens,
        temperature=temperature,
    )
    
if stream:
    for chunk in response:
        if chunk.choices and chunk.choices[0].delta.content:
            print(chunk.choices[0].delta.content, end="", flush=True)
    print()  # nova linha após streaming
else:
    print(response.choices[0].message.content)

## 4. Why Multi-Field RAG is Better: Example Queries

Using multiple fields (`short_description`, `content`, and `close_notes`) instead of just `short_description` significantly improves retrieval quality for certain types of queries. Here are examples where multi-field RAG outperforms single-field RAG:

### Example 1: Troubleshooting Steps and Solutions
**Query**: "How do I fix ZTrend crashes when saving files?"

- **Single-field (short_description only)**: May retrieve tickets about crashes, but won't have the solution steps
- **Multi-field**: Retrieves tickets with both the problem description AND the detailed troubleshooting steps from `close_notes`, providing complete answers

### Example 2: Historical Context and Resolution
**Query**: "What was the root cause and resolution for application crashes related to memory issues?"

- **Single-field**: Only finds tickets mentioning "crashes" but misses the diagnostic details and resolution steps
- **Multi-field**: Retrieves tickets with full context from `content` (initial problem description) and `close_notes` (diagnostic findings and resolution), enabling comprehensive answers

### Example 3: Pattern Recognition Across Problem-Solution Pairs
**Query**: "What are common solutions for software crashes that involve configuration files?"

- **Single-field**: Can identify crash-related tickets but can't see the solutions
- **Multi-field**: Can match both problem patterns (from `short_description`/`content`) and solution patterns (from `close_notes`), enabling identification of recurring problem-solution patterns

### Example 4: Detailed Technical Information
**Query**: "Show me tickets where log file analysis revealed the issue"

- **Single-field**: May miss tickets where log analysis is only mentioned in `content` or `close_notes`
- **Multi-field**: Captures technical details from all fields, ensuring comprehensive retrieval of relevant tickets

### Example 5: End-to-End Ticket Understanding
**Query**: "Find tickets where the customer reported a problem, diagnostics were performed, and the issue was resolved by reinstalling software"

- **Single-field**: Can't capture the full narrative flow from problem → diagnosis → solution
- **Multi-field**: Preserves the complete ticket lifecycle, enabling retrieval based on complex multi-stage scenarios

**Key Insight**: Multi-field RAG is especially powerful for queries that require understanding both the problem AND the solution, or queries that need to match patterns across different stages of the ticket lifecycle.


## Key Takeaways
This notebook demonstrated how to set up and use the built-in RAG tool for ingesting user-provided documents in a vector database and utilizing them during inference via direct retrieval. 

Key points:
- **Multi-field content**: We combined `short_description`, `content`, and `close_notes` fields to create richer document representations, improving the quality of retrieval and context understanding.
- **Metadata preservation**: Other fields from the dataset are stored as metadata, allowing for filtering and additional context during retrieval.
- **Vector database integration**: The documents are chunked and indexed into ChromaDB using Llama Stack's RAG tool, enabling semantic search over the ticket data.
- **Query advantages**: As shown in Section 4, multi-field RAG excels at queries requiring both problem and solution context, pattern recognition across ticket lifecycle stages, and comprehensive technical information retrieval.

Now that we've seen how easy it is to implement RAG with Llama Stack, We'll move on to building a simple agent with Llama Stack next in our [Simple Agents](./Level2_simple_agent_with_websearch.ipynb) notebook.